In [1]:
# Modify the file 00_setup.py to define input/output file paths on your system
# The information in 00_setup.py will be used across notebooks
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "./00_setup.py").load_module()

## Neural Network With NAICS Embedding Layer
Add NAICS embedding to model.  Only add the baseline NAICS (not higher levels in the hierarchy).  Do not do any data manipulation, so unseen codes get mapped to an input that is not in training

Use the dataset with no missing values. 

*This script takes about 2 hours on my MacBook Air*

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
from pathlib import Path
import importlib, pickle
from sklearn import model_selection

In [4]:
import pandas as pd
import os
import re

import keras
from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, feature_extraction, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
from tensorflow.keras.layers import Dense, Dropout, Input, Embedding, \
    Concatenate, Reshape, concatenate, Flatten
import tensorflow.keras.metrics as km

In [6]:
from sba_nn.sba_nn import sbnn_metrics 

## Input Data

In [7]:
sba_loans = pd.read_parquet(Path(setup.temp_path).joinpath('01_DATA_transformed_nomiss.parquet'))

In [8]:
with open(Path(setup.temp_path).joinpath('01_DATA_features.pkl'), 'rb') as fin:
    imputer_features = pickle.load(fin)

In [9]:
with open(Path(setup.temp_path).joinpath('01_DATA_naics_max_encodings.pkl'), 'rb') as fin:
    naics_max_levels= pickle.load(fin)
naics_max_levels

{'NAICS': 1170,
 'NAICS_5': 764,
 'NAICS_4': 345,
 'NAICS_3': 107,
 'NAICS_sector': 21}

##### Features

In [10]:
# Numeric features
features_numeric = [f for f in imputer_features if 'NAICS' not in f]
features_numeric_len = len(features_numeric)
print(features_numeric)

['NoEmp', 'CreateJob', 'LowDoc', 'DisbursementGross', 'new_business', 'urban_flag', 'franchise_flag', 'missingindicator_LowDoc', 'missingindicator_new_business', 'missingindicator_urban_flag']


In [11]:
# NAICS feature - just bas
features_naics = ['NAICS']
features_naics_max_levels  = [naics_max_levels[n] for n in features_naics]
features_naics_emb_dim = [setup.nn_naics_embed_size_dict[n] for n in features_naics]

In [12]:
print(features_naics_max_levels)
print(features_naics_emb_dim)

[1170]
[8]


##### Datasets for train, validation

In [13]:
X = sba_loans[['dset', 'LoanNr_ChkDgt'] + features_numeric].set_index('LoanNr_ChkDgt').sort_index()

In [14]:
X_train = X[X['dset'] == 'train'].drop(columns='dset')
y_train = sba_loans[sba_loans['dset'] == 'train'].set_index('LoanNr_ChkDgt').sort_index()['target']
print(f'training X: {X_train.shape}, y:{y_train.shape}')

training X: (446848, 10), y:(446848,)


In [15]:
X_val = X[X['dset'] == 'val'].drop(columns='dset')
y_val = sba_loans[sba_loans['dset'] == 'val'].set_index('LoanNr_ChkDgt').sort_index()['target']
print(f'val X: {X_val.shape}, y:{y_val.shape}')

val X: (95604, 10), y:(95604,)


In [16]:
base_thresh = y_train.mean()
print(base_thresh)

0.20230592953308507


In [17]:
X_naics = [sba_loans[['dset', 'LoanNr_ChkDgt', n]].set_index('LoanNr_ChkDgt').sort_index() \
           for n in features_naics]

In [18]:
X_naics_train = [Xn[Xn['dset'] == 'train'].drop(columns='dset') for Xn in X_naics]
X_naics_val =  [Xn[Xn['dset'] == 'val'].drop(columns='dset') for Xn in X_naics]

## Function to create model

In [19]:
def create_model(n_feat_base,
                 naics_max_levels,
                 naics_embedding_dim,
                 naics_embedding_names = None,
                 hidden_size = setup.nn_layer_sizes,
                 activation='tanh', 
                 lr=setup.nn_learning_rate,
                 opt_func = setup.nn_optimizer, 
                 dropout = setup.nn_dropout):
    
    # NAICS (embedding) layers
    naics_in_layers = []
    naics_embedding_layers = []
    if naics_embedding_names is None:
        naics_embedding_names = [f'flat_naics_{x:02d}' for x in range(len(naics_max_levels))]
    
    for i in range(len(naics_max_levels)):
        input_layer = Input(shape=(1,), name=f'input_naics_{i:02d}')
        embedding_layer = Embedding(input_dim=naics_max_levels[i], 
                                    output_dim=naics_embedding_dim[i],
                                    name=f'emb_naics_{i:02d}')(input_layer)
        flatten_layer = Flatten(name = naics_embedding_names[i])(embedding_layer)
        
        naics_in_layers.append(input_layer)
        naics_embedding_layers.append(flatten_layer)
    
    # Concatenate all flattened embeddings
    concatenated_embeddings = concatenate(naics_embedding_layers)
    
    # Numeric inputs
    numeric_in_layer = Input(shape=(n_feat_base,), name='input_numeric')
        
    # Concatenate layers so far
    x = concatenate(naics_embedding_layers + [numeric_in_layer], name='input_concat')
    
    n_layers = len(hidden_size)
    
    for i in range(n_layers):
        x = Dense(hidden_size[i],activation=activation,
                  name=f'layer_{i:02d}')(x)
        x = Dropout(dropout, name=f'dropout_{i:02d}')(x)
    output = Dense(1, activation='sigmoid', name='output')(x)
    
    model = Model(inputs=[numeric_in_layer] + naics_in_layers, outputs=output)
    
    # Compile model
    optimizer = opt_func(learning_rate=lr)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, 
                  metrics=[km.AUC(curve='PR'), km.AUC(curve='ROC')])
    return model

## Create, fit model

In [20]:
this_model = create_model(features_numeric_len,
                         features_naics_max_levels,
                         features_naics_emb_dim,
                         naics_embedding_names = features_naics)

2024-05-16 21:49:01.805856: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-05-16 21:49:01.805884: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-16 21:49:01.805891: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-16 21:49:01.805943: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-16 21:49:01.805962: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [21]:
this_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_naics_00 (InputLayer  [(None, 1)]                  0         []                            
 )                                                                                                
                                                                                                  
 emb_naics_00 (Embedding)    (None, 1, 8)                 9360      ['input_naics_00[0][0]']      
                                                                                                  
 NAICS (Flatten)             (None, 8)                    0         ['emb_naics_00[0][0]']        
                                                                                                  
 input_numeric (InputLayer)  [(None, 10)]                 0         []                        

In [ ]:
this_history = this_model.fit([X_train] + X_naics_train, y_train,
                              validation_data=([X_val] + X_naics_val, y_val),
                              batch_size=setup.nn_batch_size,
                              epochs=setup.nn_epochs)

Epoch 1/20


2024-05-16 21:49:02.559735: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13962/13964 [============================>.] - ETA: 0s - loss: 0.4625 - auc: 0.3365 - auc_1: 0.6996

2024-05-16 21:51:50.637877: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13964/13964 [==============================] - 193s 14ms/step - loss: 0.4625 - auc: 0.3365 - auc_1: 0.6996 - val_loss: 0.4539 - val_auc: 0.3697 - val_auc_1: 0.7219
Epoch 2/20
13964/13964 [==============================] - 219s 16ms/step - loss: 0.4547 - auc: 0.3586 - auc_1: 0.7163 - val_loss: 0.4513 - val_auc: 0.3793 - val_auc_1: 0.7278
Epoch 3/20
13964/13964 [==============================] - 202s 14ms/step - loss: 0.4523 - auc: 0.3662 - auc_1: 0.7212 - val_loss: 0.4502 - val_auc: 0.3828 - val_auc_1: 0.7296
Epoch 4/20
13964/13964 [==============================] - 210s 15ms/step - loss: 0.4510 - auc: 0.3700 - auc_1: 0.7239 - val_loss: 0.4504 - val_auc: 0.3841 - val_auc_1: 0.7297
Epoch 5/20
13964/13964 [==============================] - 208s 15ms/step - loss: 0.4503 - auc: 0.3726 - auc_1: 0.7251 - val_loss: 0.4499 - val_auc: 0.3846 - val_auc_1: 0.7305
Epoch 6/20
13964/13964 [==============================] - 208s 15ms/step - loss: 0.4497 - auc: 0.3742 - auc_1: 0.7266 - val_loss: 0.4498

In [ ]:
this_history_df = pd.DataFrame(this_history.history)

In [ ]:
# Rename columns
try:
    this_history_df.columns = ['_'.join(c.split('_')[0:-1])  \
                               if re.search(r'_\d+$', c) else c for c in this_history_df.columns]
except:
    pass
try:
    cur_col = list(this_history_df.columns)
    this_history_df.columns = [cur_col[0]] + \
        [f'{cur_col[i]}_roc'  if (cur_col[i] == cur_col[i-1]) and 'auc'in cur_col[i] \
         else cur_col[i] for i in range(1, len(cur_col))]
except:
    pass

In [ ]:
this_history_df.columns

In [ ]:
this_history_df.to_csv(Path(setup.temp_path).joinpath('03_REPORT_fit_history.csv'))

In [ ]:
this_history_df[['loss', 'val_loss']].plot()

In [ ]:
this_history_df[['auc', 'val_auc']].plot()

In [ ]:
this_history_df[['auc_roc', 'val_auc_roc']].plot()

In [ ]:
this_model.save(Path(setup.temp_path).joinpath('03_DATA_model.keras'),save_format='tf')

## Predictions on all data

In [ ]:
X_naics = [sba_loans[['dset', 'LoanNr_ChkDgt', n]].set_index('LoanNr_ChkDgt').sort_index() \
           for n in features_naics]

In [ ]:
all_predictions = this_model.predict([X.drop(columns='dset')] +
                                     [Xn.drop(columns='dset') for Xn in X_naics])

In [ ]:
all_predictions_df = pd.DataFrame(all_predictions, index=X.index) \
    .set_axis(['predict_prob'], axis=1) \
    .reset_index() \
    .merge(sba_loans[['target', 'LoanNr_ChkDgt', 'dset', 'dset_naics_holdout', 'NAICS']], 
           on='LoanNr_ChkDgt')

In [ ]:
all_predictions_df[['predict_prob', 'target']].corr(method='spearman')

##### Threshold Tune & Binary Predictions
Using training probability predictions

In [ ]:
all_pred_train = all_predictions_df[all_predictions_df['dset'] == 'train']

In [ ]:
thresh_tune_data = sbnn_metrics.get_f1_frame(all_pred_train['target'], 
                                        all_pred_train['predict_prob'])

In [ ]:
thresh_tune_data.sort_values('f1', ascending=False, inplace=True)
thresh_tune_data.head(3)

In [ ]:
best_thresh = thresh_tune_data['thresh'].iloc[0]
best_thresh

##### Append binary predictions to probability predictions

In [ ]:
all_predictions_df['predict_bin'] = sbnn_metrics.get_binary_predictions(all_predictions_df['predict_prob'], best_thresh)

In [ ]:
all_predictions_df['predict_bin'].value_counts(normalize=True, dropna=False)

In [ ]:
all_predictions_df.to_parquet(Path(setup.temp_path).joinpath('03_DATA_predictions.parquet'))

## Metrics

In [ ]:
metrics_dset_df = all_predictions_df.groupby('dset') \
    .apply(lambda x: sbnn_metrics.dset_metrics(x.target, x.predict_bin, x.predict_prob)) \
    .reset_index()
metrics_test_df = all_predictions_df[all_predictions_df['dset'] == 'test'] \
    .groupby(['dset', 'dset_naics_holdout']) \
    .apply(lambda x: sbnn_metrics.dset_metrics(x.target, x.predict_bin, x.predict_prob)) \
    .reset_index()
metrics_df = pd.concat([metrics_dset_df, metrics_test_df])
metrics_df.to_csv(Path(setup.temp_path).joinpath('03_REPORT_metrics.csv'), index=True)
metrics_df